<a href="https://colab.research.google.com/github/INFINITY-RUBER/Curso_de_Tensorflow-2.0_Guia_completa_para_el_Nuevo_Tensorflow/blob/master/Collab_9_Validaci%C3%B3n_de_datos_con_TensorFlow_Data_Validation_(TFDV).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Paso 1:Instalar todas las dependencias y configurar el entorno

In [1]:
!pip install tensorflow-data-validation

In [2]:
!apt-get install python-dev python-snappy

Reading package lists... Done
Building dependency tree       
Reading state information... Done
python-dev is already the newest version (2.7.15~rc1-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
The following NEW packages will be installed:
  python-snappy
0 upgraded, 1 newly installed, 0 to remove and 35 not upgraded.
Need to get 10.8 kB of archives.
After this operation, 39.9 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 python-snappy amd64 0.5-1.1build2 [10.8 kB]
Fetched 10.8 kB in 0s (52.0 kB/s)
Selecting previously unselected package python-snappy.
(Reading database ... 144487 files and directories currently installed.)
Preparing to unpack .../python-snappy_0.5-1.1build2_amd64.deb ...
Unpacking python-snappy (0.5-1.1build2) ...
Setting up python-snappy (0.5-1.1build2) ...


In [3]:
#!pip install -q tensorflow_data_validation

## Paso 2: Importar las librerías para el proyecto

In [3]:
import pandas as pd
import tensorflow as tf
import tensorflow_data_validation as tfdv

from __future__ import print_function

## Paso 3: Análisis descriptivo del dataset


In [5]:
dataset = pd.read_csv("pollution-small.csv")

In [6]:
dataset.shape

(2188, 5)

In [7]:
training_data = dataset[:1600]

In [8]:
training_data.describe()

,pm10,no2,so2,soot
count,1600.000000,1600.000000,1600.000000,1600.000000
mean,49.656494,30.980519,16.229981,21.551956
std,35.211906,12.400788,10.621896,12.127354
min,6.380000,9.740000,4.010000,6.000000
25%,28.345000,22.567500,9.777500,14.400000
50%,38.835000,28.715000,13.275000,18.630000
75%,58.050000,36.370000,19.282500,24.072500
max,277.250000,138.010000,123.130000,107.650000


In [10]:
test_set = dataset[1600:]


In [11]:
test_set.describe()

,pm10,no2,so2,soot
count,588.000000,588.000000,588.000000,588.000000
mean,44.648248,37.296922,13.605170,18.441310
std,28.992087,10.940050,5.098944,6.596459
min,11.900000,15.070000,4.990000,8.000000
25%,28.337500,29.217500,10.122500,14.410000
50%,35.555000,35.815000,12.345000,17.090000
75%,50.812500,43.872500,15.855000,20.962500
max,273.770000,106.030000,38.030000,87.210000


## Paso 3b: Análisis descriptivo y validación con TFDV

### Generar análisis descriptivo del dataset


In [12]:
train_stats = tfdv.generate_statistics_from_dataframe(dataframe=training_data)
#train_stats = tfdv.generate_statistics_from_tfrecord(data_location= training_data)

### Inferir el esquema 

In [14]:
schema = tfdv.infer_schema(statistics=train_stats)

In [15]:
tfdv.display_schema(schema)

,Type,Presence,Valency,Domain
Feature name,,,,
'Date',BYTES,required,,-
'pm10',FLOAT,required,,-
'no2',FLOAT,required,,-
'so2',FLOAT,required,,-
'soot',FLOAT,required,,-


### Calcular los estadísticos  descriptivos del conjunto de testing


In [18]:
test_stats = tfdv.generate_statistics_from_dataframe(dataframe=test_set)


In [19]:
schema2 = tfdv.infer_schema(statistics=test_stats)
tfdv.display_schema(schema2)

,Type,Presence,Valency,Domain
Feature name,,,,
'Date',BYTES,required,,-
'pm10',FLOAT,required,,-
'no2',FLOAT,required,,-
'so2',FLOAT,required,,-
'soot',FLOAT,required,,-


## Paso 4: Comparar estadísticos del conjunto de test con Schema

### Buscar anomalías en los nuevos datos

In [22]:
anomalies = tfdv.validate_statistics(statistics=test_stats, schema=schema)

### Mostrar todas las anomalías detectadas

- Número entero mayor que  10
- Tipo STRING cuando se esperaba un tipo INT
- Tipo FLOAT cuando se esperaba un tipo INT
- Número entero menor que 0

In [23]:
tfdv.display_anomalies(anomalies) # informa si hay anomalias en la informacion

### Nuevos datos CON anomalías

In [25]:
test_set_copy = test_set.copy()
test_set_copy.head()

,Date,pm10,no2,so2,soot
1600,5/20/2013,24.55,32.39,11.62,20.13
1601,5/21/2013,22.03,30.34,13.29,20.56
1602,5/22/2013,17.36,24.06,12.15,20.07
1603,5/23/2013,16.25,26.81,11.19,19.20
1604,5/24/2013,22.51,26.01,12.81,20.73


In [26]:
test_set_copy.drop("soot", axis=1, inplace=True) # elimino la columna "soot" implicita en ella misma
test_set_copy.head()

,Date,pm10,no2,so2
1600,5/20/2013,24.55,32.39,11.62
1601,5/21/2013,22.03,30.34,13.29
1602,5/22/2013,17.36,24.06,12.15
1603,5/23/2013,16.25,26.81,11.19
1604,5/24/2013,22.51,26.01,12.81


### Estadísticos basados en datos con anomalías

In [27]:
test_set_copy_stats = tfdv.generate_statistics_from_dataframe(dataframe=test_set_copy)

In [28]:
anomalies_new = tfdv.validate_statistics(statistics=test_set_copy_stats, schema=schema)

In [30]:
tfdv.display_anomalies(anomalies_new)

/usr/local/lib/python3.6/dist-packages/tensorflow_data_validation/utils/display_util.py:173: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('max_colwidth', -1)


,Anomaly short description,Anomaly long description
Feature name,,
'soot',Column dropped,Column is completely missing


## Paso 5: Preparar el esquema para subir a producción

In [31]:
schema.default_environment.append("TRAINING")
schema.default_environment.append("SERVING")

### Eliminar la columna objetivo del esquema de producción

In [32]:
tfdv.get_feature(schema, "soot").not_in_environment.append("SERVING")
# .get_feature(schema, "soot")  ==>> Del esquema me devuelve la columna "soot" 
# .not_in_environment ==>> funcio de tranforamcion para eliminarla del entorno
# .append("SERVING") ==>> apendizo "SERVING"
## tfdv.get_feature(schema, "soot").not_in_environment.append("SERVING") == La Columna "soot" no estara en el entordo del "SERVING"

### Comprobar anomalías entre el entorno del servidor y nuevos datos entrantes

In [33]:
serving_env_anomalies = tfdv.validate_statistics(test_set_copy_stats, schema, environment="SERVING")

In [34]:
tfdv.display_anomalies(serving_env_anomalies)

## Paso 6: Congelar el esquema

In [35]:
tfdv.write_schema_text(schema = schema, output_path = "pollution_schema.pbtxt")

# ¿Y ahora qué más?
Hemos visto mucho temario en esta sección - desde importar el dataset a crear un esquema de TensorFlow Data Validation que detecta anomalías en tiempo real en el servidor!

Sin embargo, ¿ahora qué? ¿Qué hacemos después de guardar el esquema?

TensorFlow Data Validation (TFDV) es una pequeña parte de una tecnología más general llamada TensorFlow Extended, que se ha creado para poder llevar todo un proyecto de deep learning desde su concepción hasta su puesta a punto en producción.

El esquema que hemos guardado en el último vídeo de la sección se puede usar de muchos modos. El más común sería junto con TensorFlow Transform, parte también de la librería TensorFlow Extended, y que verás conmigo en la próxima sección ;). Esta tecnología es muy útil para pipelines desde la recogida o minado del dato hasta su almacenamiento para análisis posterior, ya que TensorFlow Transform conoce el tipo de dato que espera basándose en las estadísticas generadas en el esquema.

Hay también otra forma muy útil de usar el esquema creado, y es usando la visualización de datos antes del periodo de inferencia.



Te dejo aquí un par de links adicionales donde podrás leer y aprender más acercan TFDV y cómo usarlo en implementaciones end-to-end:



1. Un gran post del blog Towards Data Science donde se explica como usar la parte de visualización de datos de TFDV y el proceso ingenieril necesario en mucho más detalle:
https://towardsdatascience.com/hands-on-tensorflow-data-validation-61e552f123d7

2. Tutorial original de TensorFlow sobre como implementar el TFDV de principio hasta el fin:
https://www.tensorflow.org/tfx/guide/tfdv

3. TensorFlow Data Validation - Pipeline de validación de datos personalizada:
https://www.tensorflow.org/tfx/data_validation/get_started

In [43]:
tfdv.display_schema(schema=schema)

,Type,Presence,Valency,Domain
Feature name,,,,
'Date',BYTES,required,,-
'pm10',FLOAT,required,,-
'no2',FLOAT,required,,-
'so2',FLOAT,required,,-
'soot',FLOAT,required,,-
